# FABEMD

In [29]:
import numpy as np
from scipy.ndimage import maximum_filter, minimum_filter, uniform_filter


def get_local_extrema(image, window_size=3):
    """Get local maxima and minima maps for a given image."""
    max_map = image == maximum_filter(image, size=window_size)
    min_map = image == minimum_filter(image, size=window_size)
    return max_map, min_map


def apply_order_statistic_filter(image, extrema_map, filter_type="max", window_size=3):
    """Approximate the envelope using order statistics filters (MAX/MIN)"""
    if filter_type == "max":
        envelope = maximum_filter(image, size=window_size)
    elif filter_type == "min":
        envelope = minimum_filter(image, size=window_size)
    else:
        raise ValueError("filter_type should be either 'max' or 'min'")

    return np.where(extrema_map, envelope, image)


def smooth_envelope(envelope, smooth_window_size=3):
    """Smooth the envelope with an averaging filter."""
    return uniform_filter(envelope, size=smooth_window_size)


def calculate_mean_envelope(upper_envelope, lower_envelope):
    """Calculate the mean envelope."""
    return (upper_envelope + lower_envelope) / 2


def calculate_standard_deviation(FTj, FTj_next):
    """Calculate the standard deviation used for BIMF criteria checking."""
    return np.sqrt(np.sum((FTj_next - FTj) ** 2) / np.sum(FTj**2))


def fabemd(image, max_iterations=10, threshold=0.2, initial_window_size=3):
    """
    Perform FABEMD decomposition on an input image.

    Args:
        image (np.ndarray): Input image (2D array).
        max_iterations (int): Maximum iterations for BIMF extraction.
        threshold (float): Threshold for standard deviation to accept BIMF.
        initial_window_size (int): Initial window size for finding extrema.

    Returns:
        list: A list of extracted BIMFs.
        np.ndarray: Residue of the decomposition.
    """
    residual = image.astype(float)
    BIMFs = []

    # Iterate to extract each BIMF
    while True:
        FTj = residual.copy()
        window_size = initial_window_size

        for j in range(max_iterations):
            # Step 1: Find local maxima and minima
            max_map, min_map = get_local_extrema(FTj, window_size=window_size)

            # Step 2: Estimate upper and lower envelopes using MAX/MIN filters
            upper_envelope = apply_order_statistic_filter(
                FTj, max_map, filter_type="max", window_size=window_size
            )
            lower_envelope = apply_order_statistic_filter(
                FTj, min_map, filter_type="min", window_size=window_size
            )

            # Step 3: Smooth the envelopes
            upper_envelope = smooth_envelope(
                upper_envelope, smooth_window_size=window_size
            )
            lower_envelope = smooth_envelope(
                lower_envelope, smooth_window_size=window_size
            )

            # Step 4: Calculate the mean envelope
            mean_envelope = calculate_mean_envelope(upper_envelope, lower_envelope)

            # Step 5: Update FTj for the next iteration
            FTj_next = FTj - mean_envelope
            SD = calculate_standard_deviation(FTj, FTj_next)

            # Check if the BIMF conditions are met
            if SD < threshold:
                BIMFs.append(FTj_next)
                residual -= FTj_next
                break
            else:
                FTj = FTj_next
                window_size += 2  # Optionally adjust window size with each iteration

        # Stop if the residual has fewer than 3 extrema points
        max_map, min_map = get_local_extrema(residual)
        if np.sum(max_map) < 3 or np.sum(min_map) < 3:
            break

    return BIMFs, residual


# Usage:
# Assuming `input_image` is the (4096, 3255) grayscale image
# FABEMD decomposition on an example large image
# BIMFs, residue = fabemd(input_image)

In [ ]:
import cv2

image_path = "image_sample/1-IMA-01B_Thorax_AP.tiff"

# Load the image
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# describe the image
print("Image shape:", image.shape)

# Resize the image to a smaller size for faster processing
# using 0.1x scaling factor
image = cv2.resize(image, (0, 0), fx=0.1, fy=0.1)

# describe the resized image
print("Resized image shape:", image.shape)

# Perform FABEMD decomposition
BIMFs, residue = fabemd(image)

# Display the results
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
plt.subplot(2, 3, 1)
plt.imshow(image, cmap="gray")
plt.title("Original Image")
plt.axis("off")

for i, bimf in enumerate(BIMFs):
    plt.subplot(2, 3, i + 2)
    plt.imshow(bimf, cmap="gray")
    plt.title(f"BIMF {i + 1}")
    plt.axis("off")

plt.subplot(2, 3, 6)
plt.imshow(residue, cmap="gray")
plt.title("Residue")
plt.axis("off")

Image shape: (4096, 3255)
Resized image shape: (410, 326)
